# Discogs

In [1]:
######################################################
# Juypter
######################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


######################################################
# General
######################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from ioUtils import getFile, saveFile
from fileUtils import getFileBasics, getBasename
from webUtils import getWebData, getHTML, getURL
from time import sleep
import time
from multiprocessing import Pool


######################################################
# Music Stuff
######################################################
from dbBase import dbBase
from multiArtist import multiartist
from myMusicDBMap import myMusicDBMap
from mainDB import mainDB
from parseDBArtistsData import parseDBArtistsData


######################################################
# Versions
######################################################
import sys
print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-08-20 18:58:29.517390


# Initialize

In [2]:
mdb = myMusicDBMap(debug=True)

Creating myMusicDBMap()
   Loading my music db map: /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Found 4177 artist entries
Counter({'Discogs': 4010, 'AllMusic': 3952, 'MusicBrainz': 3743, 'LastFM': 2860, 'DatPiff': 536, 'RockCorner': 481, 'AceBootlegs': 176, 'CDandLP': 173, 'RateYourMusic': 56, 'MusicStack': 5, 'MetalStorm': 0})
Counter({'Discogs': 4010, 'AllMusic': 3952, 'MusicBrainz': 3743, 'LastFM': 2860, 'DatPiff': 536, 'RockCorner': 481, 'AceBootlegs': 176, 'CDandLP': 173, 'RateYourMusic': 56, 'MusicStack': 5, 'MetalStorm': 0})


In [3]:

multiDelimArtists=open("../multiartist/multiDelimArtists.dat").readlines()
multiDelimArtists = [x.replace("\n", "") for x in multiDelimArtists]
multiDelimArtists[:3]

mulArts  = multiartist(cutoff=0.9, discdata=None, exact=False)
mulArts.setKnownMultiDelimArtists(multiDelimArtists)

Adding 319 known multi delim artists.


In [4]:
mulArts.getArtistNames("Drake & Future", debug=True)
print(len(mulArts.knownMultiDelimArtists))
#sorted(mulArts.knownMultiDelimArtists)

Getting Artist Names For Drake & Future
Finding Artists Within: Drake & Future
1 	 Drake & Future ===> {'&': ['Drake', 'Future']}
2 	 Future ===> {}
Adding Future. Sum is 0
2 	 Drake ===> {}
Adding Drake. Sum is 1
Combining Results
All Artists:   {'Future': True, 'Drake': True}
Known Artists: {'Future', 'Drake'}
319


# Search

In [5]:
mdb.getArtists()

['"Weird Al" Yankovic',
 "'68 Comeback",
 "'68 Comeback Special",
 '*NSYNC',
 '-123 min.',
 '1-2 Japanese',
 '1.6 Band',
 '10,000 Maniacs',
 '10cc',
 '112',
 '12 Lb. Test',
 '13th Floor Elevators',
 '16',
 '2 Chainz',
 '2 Unlimited',
 '25 Ta Life',
 '2Gether',
 '2Pac',
 '2nd Avenue',
 '3 Doors Down',
 '3-Way Cum',
 '311',
 '38 Special',
 '3OH!3',
 '3T',
 '3X Krazy',
 '40 Grit',
 '454 Big Block',
 '5 Seconds Of Summer',
 '50 Cent',
 '59 Times The Pain',
 '69 Boyz',
 '69 Tribe',
 '7 Angels 7 Plagues',
 '7000 Dying Rats',
 '764-Hero',
 '9 Shocks Terror',
 '9-Iron',
 '90 Day Men',
 '98 Degrees',
 ':wumpscut:',
 'A Flock Of Seagulls',
 'A Global Threat',
 'A Great Big World',
 'A Tribe Called Quest',
 'A*Teens',
 'A-Mafia',
 'A-Set',
 'A.D.O.R.',
 'ABBA',
 'ABC',
 'AC-DC',
 'AFI',
 'AIR',
 'AJR',
 'ATB',
 'Aaron Neville',
 'Abdullah',
 'Abigor',
 'Abominator',
 'Above The Law',
 'Abscess',
 'Absolution',
 'Absurdus',
 'Abwärts',
 'Accept',
 'Ace Of Base',
 'Acker Bilk',
 'Action League',
 '

In [6]:
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata
#dbdata

Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   datpiff   =========================
=========================   acebootlegs   =========================
=========================   cdandlp   =========================
=========================   rateyourmusic   =========================
=========================   musicstack   =========================
  Creating Database Records for Discogs
  Creating Database Records for AllMusic
  Creating Database Records for MusicBrainz
  Creating Database Records for LastFM
  Creating Database Records for RockCorner
  Creating Database Records for DatPiff
  Creating Database Records for AceBootlegs
  Creating Databas

In [10]:
ifile = "unmatchedArtists4.p"
#noMatches = getFile(ifile)
#noMatches = ['Ava Max']

noMatches = ['The Jive Five With Joe Rene And Orchestra',
 'The Singing Nun (Soeur Sourire)',
 'Little Stevie Wonder',
 'Jimmy Gilmer And The Fireballs',
 '? (Question Mark) & The Mysterians',
 'Grand Funk',
 'The Independents',
 'Bo Donaldson And The Heywoods',
 'Faith, Hope And Charity',
 'Meco',
 'LTD',
 'Schuyler, Knobloch And Bickhardt',
 'Samuele',
 "Compton's Most Wanted",
 "UMC's",
 'DRS',
 'Michael Peterson',
 'No Good',
 'INFINITE',
 'tUnE-YaRdS',
 'CAAMP',
 'Getz',
 'The Mike Curb Congregation',
 'The Dixie Flyers',
 'John Ford Coley',
 'DA Bulldogs',
 'ED O.G',
 'BBD',
 'KRS-1',
 'NAS Escobar',
 'Sheryl Sal',
 'Sons Of The Desert',
 'DSD',
 'Paul McCoy',
 "Brandy 'Ms. B.' Hambrick",
 'Bluenote Cafe']

noMatches = ["Joe Rene And Orchestra", "Sheryl Sal", "Salt", "Brandy Hambrick"]
noMatches = ["José José"]

noMatches = ['John Williams',
 'George Benson',
 'The Cooper Temple Clause',
 'Emma',
 'The Bay City Rollers',
 'Macdonald Bros',
 'LSG',
 '8Ball & MJG',
 'John Cena & tha Trademarc',
 'Hannah',
 'k.d. lang',
 'Yanni',
 'Sharon Jones & The Dap-Kings',
 'Jaron & The Long Road To Love',
 'Maranatha! Praise Band',
 'Ghost B.C.',
 'Mandisa',
 'Aer',
 '††† (Crosses)',
 'SoMo',
 'The Vamps',
 'BØRNS',
 '3rdeyegirl']

noMatches = ['Questian Mark & The Mysterians',
 'Faith, Hope And Charity',
 "Compton's Most The Wanted",
 'Ghost B.C.',
 'Joe Rene And Orchestra',
 'Sammy Davis, Jr.',
 'The McCampbell Brothers',
 'La Arrolladora Banda el Limon de Rene Camacho']

noMatches = ["Ghost"]

noMatches = ['Don Toliver', 'Gabby Barrett', 'Jack Harlow', 'Jane Siberry', 'Lizzo', 'Mario Lanza', 'Nat King Cole', 'Noah Cyrus', '21 Savage', 'Perry Como', 'Powfu', 'Saint Jhn', 'Sam Hunt', 'Shoreline Mafia', 'YoungBoy Never Broke Again']
#noMatches = ['Mario Lanza', 'Nat King Cole', 'Noah Cyrus', '21 Savage', 'Perry Como', 'Powfu', 'Saint Jhn', 'Sam Hunt', 'Shoreline Mafia', 'YoungBoy Never Broke Again']


print(len(noMatches))
dbs = ['Discogs', 'MusicBrainz', 'LastFM']
#dbs = ['AllMusic', 'Discogs', 'MusicBrainz', 'LastFM']
#dbs = ['AllMusic']
#dbs = ['AllMusic', 'Discogs', 'MusicBrainz', 'LastFM']
#dbs = ['MusicBrainz', 'LastFM']

#noMatches=['Go-A', 'PM', 'Qango']

maindb = mainDB(mdb=mdb, create=True, debug=True)
pdb = parseDBArtistsData(maindb=maindb)

from time import sleep
maxI = -1
for db in dbs:
    print(db)
    for i,artist in enumerate(noMatches):
        print("="*100)
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist)
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist,'===>',db)
        try:
            dbdata[db]["Artists"].searchForArtist(artist, force=True)
        except:
            sleep(5)

15
Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   datpiff   =========================
=========================   acebootlegs   =========================
=========================   cdandlp   =========================
=========================   rateyourmusic   =========================
=========================   musicstack   =========================
  Creating Database Records for Discogs
  Creating Database Records for AllMusic
  Creating Database Records for MusicBrainz
  Creating Database Records for LastFM
  Creating Database Records for RockCorner
  Creating Database Records for DatPiff
  Creating Database Records for AceBootlegs
  Creating Data

Saving /Volumes/Piggy/Discog/artists-discogs/63/2952963.p (force=True)
  --> This file is 20.8kB.
Done. Sleeping for 2 seconds
13 / 16 	: 6 	 https://www.discogs.com/artist/730308-Alberto-Lizzio?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/8/730308.p (force=True)
  --> This file is 69.2kB.
Done. Sleeping for 2 seconds
14 / 16 	: 7 	 https://www.discogs.com/artist/1000015-St-Paul-Slim?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/15/1000015.p (force=True)
  --> This file is 18.2kB.
Done. Sleeping for 2 seconds
15 / 16 	: 7 	 https://www.discogs.com/artist/6621835-Deathdance-2?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/35/6621835.p (force=True)
  --> This file is 17.4kB.
Done. Sleeping for 2 seconds
16 / 16 	: 7 	 https://www.discogs.com/artist/6545716-Old-Crock-Band?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/16/6545716.p (force=True)
  --> This file is 16.3kB.
Done. Sleeping for 2 

12 / 26 	: 7 	 https://www.discogs.com/artist/6334133-Nat-King-Cole-Quartet?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/33/6334133.p (force=True)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
13 / 26 	: 6 	 https://www.discogs.com/artist/883367-Maria-Cole?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/67/883367.p (force=True)
  --> This file is 19.2kB.
Done. Sleeping for 2 seconds
14 / 26 	: 7 	 https://www.discogs.com/artist/1086315-Teddi-King?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/15/1086315.p (force=True)
  --> This file is 33.6kB.
Done. Sleeping for 2 seconds
15 / 26 	: 7 	 https://www.discogs.com/artist/6488100-Nat-King-Cole-Quintette?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/0/6488100.p (force=True)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
16 / 26 	: 7 	 https://www.discogs.com/artist/1716275-The-Nat-King-Cole-Session?sort=year%2Casc&limit=500
S

3 / 26 	: 7 	 https://www.discogs.com/artist/2073519-Perry-Como-With-Mixed-Chorus?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/19/2073519.p (force=True)
  --> This file is 16.3kB.
Done. Sleeping for 2 seconds
4 / 26 	: 7 	 https://www.discogs.com/artist/4456311-Perry-Como-With-Organ-And-Choir?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/11/4456311.p (force=True)
  --> This file is 16.3kB.
Done. Sleeping for 2 seconds
5 / 26 	: 7 	 https://www.discogs.com/artist/4456312-Perry-Como-And-Mixed-Chorus-And-Orchestra?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/12/4456312.p (force=True)
  --> This file is 16.5kB.
Done. Sleeping for 2 seconds
6 / 26 	: 7 	 https://www.discogs.com/artist/4456320-Perry-Como-With-Orchestra?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/20/4456320.p (force=True)
  --> This file is 17.2kB.
Done. Sleeping for 2 seconds
7 / 26 	: 7 	 https://www.discogs.com/artist/62

6 / 26 	: 7 	 https://www.discogs.com/artist/2249352-Sam-Scott-Hunter?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/52/2249352.p (force=True)
  --> This file is 22.3kB.
Done. Sleeping for 2 seconds
7 / 26 	: 7 	 https://www.discogs.com/artist/4761684-Sam-Hunt-5?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/84/4761684.p (force=True)
  --> This file is 15.3kB.
Done. Sleeping for 2 seconds
8 / 26 	: 7 	 https://www.discogs.com/artist/1757998-Samuel-James-Hunt?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/98/1757998.p (force=True)
  --> This file is 20.9kB.
Done. Sleeping for 2 seconds
9 / 26 	: 7 	 https://www.discogs.com/artist/1121340-Sami-Hunt?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Discog/artists-discogs/40/1121340.p (force=True)
  --> This file is 16.9kB.
Done. Sleeping for 2 seconds
10 / 26 	: 6 	 https://www.discogs.com/artist/526514-Samantha-Hunt?sort=year%2Casc&limit=500
Saving /Volumes/Piggy/Disco

...

24 / 30 	: 90776223222 	 https://www.last.fm/music/Powfu%2C%2BRxseboy%2B%26%2BSlipfunc/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/22/90776223222.p (force=True)
  --> This file is 24.1kB.
Done. Sleeping for 2 seconds
25 / 30 	: 9894755089 	 https://www.last.fm/music/POWFU%2Bfeat%2BBEABADOOBEE/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/89/9894755089.p (force=True)
  --> This file is 23.9kB.
Done. Sleeping for 2 seconds
26 / 30 	: 39318575416 	 https://www.last.fm/music/Powfu%2B%26%2BSleeping/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/16/39318575416.p (force=True)
  --> This file is 23.4kB.
Done. Sleeping for 2 seconds
27 / 30 	: 16326708199 	 https://www.last.fm/music/beabadoobee%2C%2BPowfu/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/99/16326708199.p (force=True)
  --> This file is 21.9kB.
Done. Sleeping for 2 seconds
28 / 30 	: 6294363531 	 https://www.last.fm/music/Powfu%2Bx%2

26 / 30 	: 9974847092 	 https://www.last.fm/music/SAINt%2BJHN%2C%2BJanelle%2BKroll/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/92/9974847092.p (force=True)
  --> This file is 23.4kB.
Done. Sleeping for 2 seconds
27 / 30 	: 61349746495 	 https://www.last.fm/music/SAINt%2BJHN%2Bfeat.%2BLenny%2BKravitz/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/95/61349746495.p (force=True)
  --> This file is 23.9kB.
Done. Sleeping for 2 seconds
28 / 30 	: 67978920905 	 https://www.last.fm/music/SAINt%2BJHN%2Bfeat.%2BA%2BBoogie%2Bwit%2Bda%2BHoodie/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/5/67978920905.p (force=True)
  --> This file is 24.1kB.
Done. Sleeping for 2 seconds
29 / 30 	: 15205393244 	 https://www.last.fm/music/SAINt%2BJHN%2C%2BJ%2BBalvin/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/44/15205393244.p (force=True)
  --> This file is 24.4kB.
Done. Sleeping for 2 seconds
30 / 30 	: 17857980

27 / 30 	: 96069771761 	 https://www.last.fm/music/Breland%2C%2BSam%2BHunt/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/61/96069771761.p (force=True)
  --> This file is 21.5kB.
Done. Sleeping for 2 seconds
28 / 30 	: 17459714429 	 https://www.last.fm/music/BRELAND%2B%28feat.%2BSam%2BHunt%29/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/29/17459714429.p (force=True)
  --> This file is 21.2kB.
Done. Sleeping for 2 seconds
29 / 30 	: 93008601739 	 https://www.last.fm/music/Sam%2BHunt%2Bfeat.%2BCarrie%2BUnderwood/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/39/93008601739.p (force=True)
  --> This file is 19.9kB.
Done. Sleeping for 2 seconds
30 / 30 	: 76815186690 	 https://www.last.fm/music/Sam%2BThe%2BHunt/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/90/76815186690.p (force=True)
  --> This file is 20.9kB.
Done. Sleeping for 2 seconds
13 / 15 	 Shoreline Mafia
13 / 15 	 Shoreline Mafia 

Saving /Volumes/Piggy/Discog/artists-lastfm/17/55218759217.p (force=True)
  --> This file is 23.7kB.
Done. Sleeping for 2 seconds
29 / 30 	: 18799520031 	 https://www.last.fm/music/SOB%2BX%2BRBE%2B%26%2BShoreline%2BMafia/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/31/18799520031.p (force=True)
  --> This file is 23.8kB.
Done. Sleeping for 2 seconds
30 / 30 	: 60180042057 	 https://www.last.fm/music/Shoreline%2BMafia%2C%2BFenix%2BFlexin%2C%2BMac%2BP%2BDawg/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/57/60180042057.p (force=True)
  --> This file is 23.1kB.
Done. Sleeping for 2 seconds
14 / 15 	 YoungBoy Never Broke Again
14 / 15 	 YoungBoy Never Broke Again ===> LastFM


===================== Searching For YoungBoy Never Broke Again =====================
1 / 30 	: 51699242034 	 https://www.last.fm/music/YoungBoy%2BNever%2BBroke%2BAgain/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/34/51699242034.p (force=True)

Saving /Volumes/Piggy/Discog/artists-lastfm/12/42613984812.p (force=True)
  --> This file is 25.1kB.
Done. Sleeping for 2 seconds
27 / 30 	: 78080385667 	 https://www.last.fm/music/Youngboy%2BNever%2BBroke%2BAgain%2C%2BLil%2BBaby%2C%2BPlies/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/67/78080385667.p (force=True)
  --> This file is 23.5kB.
Done. Sleeping for 2 seconds
28 / 30 	: 28067293189 	 https://www.last.fm/music/Vl%2BDeck%2B%26%2BYoungBoy%2BNever%2BBroke%2BAgain/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/89/28067293189.p (force=True)
  --> This file is 24.4kB.
Done. Sleeping for 2 seconds
29 / 30 	: 68667278471 	 https://www.last.fm/music/Youngboy%2BNever%2BBroke%2BAgain%2Bfeat.%2BLil%2BBaby/+albums?order=release_date
Saving /Volumes/Piggy/Discog/artists-lastfm/71/68667278471.p (force=True)
  --> This file is 24.8kB.
Done. Sleeping for 2 seconds
30 / 30 	: 43686397518 	 https://www.last.fm/music/Youngboy%2BNever%2BBroke%2BAga

# Download Artist Data

In [ ]:
artsDC.downloadArtistUnofficialFromID(144998)

In [ ]:
maindb.getBasicArtistInfo("Black Sabbath")

In [ ]:
dbdata["Discogs"]["Artists"].downloadArtistUnofficialFromID('144998', '/artist/144998-Black-Sabbath')
#/artist/144998-Black-Sabbath?type=Unofficial&subtype=Albums&filter_anv=0"
#/artist/144998-Black-Sabbath?sort=year%2Casc&limit=500&type=Unofficial&subtype=Albums&filter_anv=0&page=1

In [ ]:
retval = dbdata["Discogs"]["Artist"].getData("/Users/tgadfort/Music/Discog/artists-discogs/98/unofficial/144998.p")

In [ ]:
bsdata = getHTML("/Users/tgadfort/Music/Discog/artists-discogs/98/unofficial/144998.p")

In [ ]:
retval.get()

In [ ]:
retval.get()['artist'].get()

# Music DB

In [ ]:
https://www.discogs.com/artist/144998-Black-Sabbath?sort=year%2Casc&limit=500?type=Unofficial&subtype=Albums&filter_anv=0
https://www.discogs.com/artist/144998-Black-Sabbath?sort=year%2Casc&limit=500&type=Unofficial&subtype=Albums&filter_anv=0&page=1
    
artistIDToRef = disc.getArtistIDToRefData()

In [ ]:
myMusicAlbumIDMap = getFile("/Users/tgadfort/Documents/code/music/musicDiscogsMap.p")

In [ ]:
dR=0.2
rC=0.8

In [ ]:
from time import sleep
N = len(myMusicAlbumIDMap)
for i,(artistName,artistVals) in enumerate(myMusicAlbumIDMap.items()):
    if artistName.startswith("Levl") is False:
        continue
    artistID,discogsName = artistVals
    print(i,N,'\t',artistName)
    artsDC.searchDiscogForArtist(artistName)
    artsAM.searchAllMusicForArtist(artistName)
    artsMB.searchMusicBrainzForArtist(artistName)
    sleep(3)

In [ ]:
artistIDs = [v[0] for k,v in myMusicAlbumIDMap.items()]
arts.rmIDsFromDBs(artistIDs)

In [ ]:
arts.parseArtistModValFiles(96, force=False)
arts.parseArtistModValExtraFiles(96)

In [ ]:
files = [savename]
from glob import glob
from os.path import join
from fileUtils import getDirname
files += glob(join(getDirname(savename), "extra", "{0}-*.p".format(artistID)))
files

# Download Missing Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
from discogsBase import discogs

disc  = discogs()
albs  = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

from ioUtils import saveFile
saveFile(idata=toget, ifile="missingArtistsFromAlbums.p")

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)

from ioUtils import getFile

toget = getFile(ifile="missingArtistsFromAlbums.p")

from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            print('\n{0}/{1}'.format(ia,len(refstoget)))
            retval = arts.downloadArtistURL(url, savename, debug=True)
            sleep(3)
        except:
            continue
    else:
        print("{0} known".format(savename))
            
    #print(item)
#len(togetmap)

# Redownload Known Artist

In [ ]:
myMusicMap = getFile("/Users/tgadfort/Documents/code/music/myMusicMap.p")

In [ ]:
musicMap = myMusicMap["Ladies of Soul"]
musicMap

In [ ]:
discDCIDToRef = discDC.getArtistIDToRefData()

In [ ]:
artistID  = musicMap['Discogs']['ID']
artistRef = discDCIDToRef.get(artistID)
artsDC.downloadArtistFromID(artistID=artistID, artistRef=artistRef, force=True)

# Download Unmatched Artists

In [ ]:
matchedResults = getFile("/Users/tgadfort/Documents/code/music/multiArtists.p")

In [ ]:
searchForArtist = {"Discogs": artsDC.searchDiscogForArtist, "AllMusic": artsAM.searchAllMusicForArtist, "MusicBrainz": artsMB.searchMusicBrainzForArtist}

In [ ]:
from time import sleep
for i,(artistName,artistValues) in enumerate(matchedResults["PartiallyUnknown"].items()):
    if i <= 83:
        continue
    print("=========== {0} [{1}/{2}] ===========".format(artistName, i, len(matchedResults["PartiallyUnknown"])))
    for dbKey, dirval in artistValues.items():
        print(artistName,'\t',searchForArtist[dbKey])
        searchForArtist[dbKey](artistName)
        sleep(1)
    sleep(1)

# Missing Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
from discogsBase import discogs

disc  = discogs()
albs  = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)

from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            _, _ = clock('\n{0}/{1}'.format(ia,len(refstoget)))
            retval = arts.downloadArtistURL(url, savename, debug=True)
        except:
            continue
    else:
        print("{0} known".format(savename))
            
    #print(item)
#len(togetmap)

# Artists From Search

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
arts.searchDiscogForArtist("World of Our Own")
#arts.searchDiscogForArtist("Continental Divide")

In [ ]:
from ioUtils import getFile

In [ ]:
toGet = getFile(ifile="/Users/tgadfort/Documents/code/charts/chartCounter.p", debug=True)

In [ ]:
toGet = getFile("~/code/charts/toget.p")

In [ ]:
for i,val in enumerate(toGet):
    if i <= 206:
        continue
    print('='*30,val,'(',i,'/',len(toGet),')','='*30)
    arts.searchDiscogForArtist(val)
    print("\n\n\n")
    sleep(2)

In [ ]:
from time import sleep
vals  = ["The Souvenirs", "The Stand", f"The Suicide Kings", "The Supremes", "The Temptations", "The Tomcats", "The Tone", "The Tourists", "The Unseen", "The Ventures", "The Vikings", "The Violents", "The Wailers", "The Want", "The Washingtonians", "The Witches", "UB40", "Underdog", "Unearth", "Unk", "Unscarred", "Up Front", "Up In Arms", "Urban Legends"]
vals += ["The Pits", "The Pressure", "The Pretenders", "The Prodigy", "The Product", "The Receptionists", "The Retardos", "The Revelers", "The Romancers", "The Roots", "The Rosehips", "The Ruiners", "The Runaways", "The Scholars", "The Seeds", "The Shams", "The Sights", "The Sillies", "The Skunks", "The Snowmen"]
vals += ["The Invisible Men", "The Israelites", "The Ivory Coast", "The Killers", "The Manatees", "The Marquees", "The Masters", "The Matrix", "The Mixtures", "The Moves", "The Noisettes", "The Nomads", "The Orange Peels", "The Outcasts"]
vals += ["The Game", "The Generics", "The Gits", "The Go-Betweens", "The Goblins", "The Godz", "The Great Society", "The Haters", "The Hilltops", "The Hookers", "The Immortals", "The Impacts", "The Inspector", "The Invaders"]
vals += ["The Diamonds", "The Diplomats", "The Dramatics", "The Drifters", "The Dust Brothers", "The Eagles", "The Exceptions", "The Fiends", "The Fixtures", "The Frantics", "The Freeze", "The Frogs", "The Fumes"]
vals += ["The Cardigans", "The Casualties", "The Challengers", "The Chameleons", "The Cheaters", "The Chicken Hawks", "The Crash", "The Crusaders", "The Cure", "The Darts", "The Deuce Coupes"]
vals += ["The Beat", "The Beatles", "The Blood Brothers", "The Blow Up", "The Bollweevils", "The Brentwoods", "The Broadways", "The Brood", "The Business", "The Busy Signals"]
vals += ["You", "Young Money", "Young Pioneers", "Zero Zero"]
for val in vals:
    print('='*30,val,'='*30)
    try:
        arts.searchDiscogForArtist(val)
    except:
        continue
    print("\n\n\n")
    sleep(10)

# New Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums

disc = discogs()
albs = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    print('\n{0}/{1}'.format(ia,len(refstoget)))
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            arts.downloadArtistURL(url, savename, debug=True)
        except:
            continue
        sleep(2)
    #print(item)
#len(togetmap)

# Albums From Known People

In [ ]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

In [ ]:
maindb = mainDB(mdb=mdb, create=False)
#maindb.setDBFull() ## Do this to recreate everything
maindb.setDBKnown()
artistDBs = maindb.getKnownArtistDBs()

# Discogs

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
ad = dbArtistsDiscogs(debug=True)
#ad.searchForArtist("Dave Matthews")
#ifile="/Users/tgadfort/Music/Discog/artists-discogs/98/unofficial/144998.p"
#ifile="/Users/tgadfort/Music/Discog/artists-discogs/70/420570.p"
ifile="/Users/tgadfort/Music/Discog/artists-discogs/34/386834.p"
retval = ad.getData(ifile)
print(retval.get()['artist'].get())
#ad.searchForArtist("Niel Diamond")

# AllMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
am = dbArtistsAllMusic(debug=True)
#am.searchForArtist("Dave Matthews")
ifile="/Users/tgadfort/Music/Discog/artists-allmusic/61/0000687561.p"
#ifile="/Users/tgadfort/Music/Discog/artists-discogs/98/unofficial/144998.p"
#ifile="/Users/tgadfort/Music/Discog/artists-discogs/70/420570.p"
#ifile="/Users/tgadfort/Music/Discog/artists-discogs/34/386834.p"
retval = am.getData(ifile)
print(retval.get()['artist'].get())

# MusicBrainz

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
am = dbArtistsMusicBrainz(debug=True)
#am.searchForArtist("Dave Matthews")
ifile="/Users/tgadfort/Music/Discog/artists-musicbrainz/71/100112022548628725831976514629370881371.p"
retval = am.getData(ifile)
print(retval.get()['artist'].get())

# LastFM

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsLastFM import dbArtistsLastFM
am = dbArtistsLastFM(debug=True)
#am.searchForArtist("Dave Matthews Band")
ifile="/Users/tgadfort/Music/Discog/artists-lastfm/14/64101467014.p"
retval = am.getData(ifile)
print(retval.get()['artist'].get())

# RockCorner

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsRockCorner import dbArtistsRockCorner
am = dbArtistsRockCorner(debug=True)
#am.searchForArtist("Dave Matthews Band")
ifile="/Users/tgadfort/Music/Discog/artists-rockcorner/67/539215767.p"
retval = am.getData(ifile)
print(retval.get()['artist'].get())

# RateYourMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsRockCorner import dbArtistsRockCorner
am = dbArtistsRockCorner(debug=True)
#am.searchForArtist("Dave Matthews Band")
#ifile="/Users/tgadfort/Music/Discog/artists-rockcorner/67/539215767.p"
#retval = am.getData(ifile)
#print(retval.get()['artist'].get())

# CDandLP

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsCDandLP import dbArtistsCDandLP
am = dbArtistsCDandLP(debug=True)
#am.searchForArtist("Dave Matthews Band")
#ifile="/Users/tgadfort/Music/Discog/artists-rockcorner/67/539215767.p"
#retval = am.getData(ifile)
#print(retval.get()['artist'].get())

# AceBootlegs

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAceBootlegs import dbArtistsAceBootlegs
am = dbArtistsAceBootlegs(debug=True)
#am.searchForArtist("Dave Matthews Band")
#ifile="/Users/tgadfort/Music/Discog/artists-rockcorner/67/539215767.p"
#retval = am.getData(ifile)
#print(retval.get()['artist'].get())

# DatPiff

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDatPiff import dbArtistsDatPiff
am = dbArtistsDatPiff(debug=True)
#am.searchForArtist("Dave Matthews Band")
#ifile="/Users/tgadfort/Music/Discog/artists-rockcorner/67/539215767.p"
#retval = am.getData(ifile)
#print(retval.get()['artist'].get())

# MusicStack

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicStack import dbArtistsMusicStack
am = dbArtistsMusicStack(debug=True)
#am.searchForArtist("Dave Matthews Band")
#ifile="/Users/tgadfort/Music/Discog/artists-rockcorner/67/539215767.p"
#retval = am.getData(ifile)
#print(retval.get()['artist'].get())

In [ ]:
ad = artistsDiscogs(debug=True)

In [ ]:
ad = artistsDiscogs(debug=True)
ifile="/Users/tgadfort/Music/Discog/artists-discogs/98/unofficial/144998.p"
retval = ad.getData(ifile)
retval.get()['artist'].get()
ad.searchForArtist("Niel Diamond")

In [ ]:
retval = ad.getData(ifile)
retval.get()['artist'].get()

In [ ]:
ad.searchForArtist("Niel Diamond")

In [ ]:
from artistAM import artistAM
from discogsUtils import allmusicUtils

class artistsAllMusic(artistsBase):
    def __init__(self, debug=False):
        self.db     = "AllMusic"
        self.disc   = discogs(self.db.lower())
        self.artist = artistAM(self.disc)
        self.dutils = allmusicUtils()
        self.debug  = debug
        super().__init__(self.db, self.disc, self.artist, self.dutils, debug=debug)

        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, artistRef, page=1):
        url = "{0}/discography/all".format(artistRef)
        return url
        
        if artistRef.startswith("http"):
            return artistRef
        else:
            baseURL = self.disc.discogURL
            url     = urllib.parse.urljoin(baseURL, quote(artistRef))
            return url

            
        if artistRef.startswith("/artist/") is False:
            print("Artist Ref needs to start with /artist/")
            return None
        
        baseURL = self.disc.discogURL
        url     = urllib.parse.urljoin(baseURL, quote(artistRef))
        url     = urllib.parse.urljoin(url, "?sort=year%2Casc&limit=500") ## Make sure we get 500 entries)
        if isinstance(page, int) and page > 1:
            pageURL = "&page={0}".format(page)
            url = "{0}{1}".format(url, pageURL)
        return url

        
    
    ##################################################################################################################
    # Search Functions
    ##################################################################################################################
    def searchForArtist(self, artist):
        print("\n\n===================== Searching For {0} =====================".format(artist))
        baseURL = self.disc.discogSearchURL
        
        url = urllib.parse.urljoin(baseURL, "{0}{1}".format("artists/", quote(artist)))
                  
                    
        ## Download data
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False

        
        ## Parse data
        bsdata = getHTML(data)
        
        artistDB  = {}
        
        uls = bsdata.findAll("ul", {"class": "search-results"})
        for ul in uls:
            lis = ul.findAll("li", {"class": "artist"})
            for li in lis:
                divs = li.findAll("div", {"class": "name"})
                for div in divs:
                    link     = div.find("a")
                    href     = link.attrs['href']
                    tooltip  = link.attrs['data-tooltip']
                    try:
                        from json import loads
                        tooltip = loads(tooltip)
                        artistID = tooltip['id']
                    except:
                        artistID = None
            
                    if artistDB.get(href) is None:
                        artistDB[href] = {"N": 0, "Name": artist}
                    artistDB[href]["N"] += 1
        
    
        if self.debug:
            print("Found {0} artists".format(len(artistDB)))
                
        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1
        
            discID   = self.dutils.getArtistID(href)
            url      = self.getArtistURL(href)
            savename = self.getArtistSavename(discID)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url)
            
            if isFile(savename):
                continue

            self.downloadArtistURL(url, savename)

In [ ]:
am = artistsAllMusic(debug=True)
#am.searchForArtist("Niel Diamond")

In [ ]:
from artistMB import artistMB
from discogsUtils import musicbrainzUtils
from hashlib import md5

class artistsMusicBrainz(artistsBase):
    def __init__(self, debug=False):
        self.db     = "MusicBrainz"
        self.disc   = discogs(self.db.lower())
        self.artist = artistMB(self.disc)
        self.dutils = musicbrainzUtils()
        self.debug  = debug
        super().__init__(self.db, self.disc, self.artist, self.dutils, debug=debug)

        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, artistRef, page=1):
        baseURL = self.disc.discogURL
        url     = urllib.parse.urljoin(baseURL, artistRef)
        return url

        
    ##################################################################################################################
    # Search Functions
    ##################################################################################################################
    def searchForArtist(self, artist):
        print("\n\n===================== Searching For {0} =====================".format(artist))
        baseURL = "https://musicbrainz.org/"
        extra = "search?query={0}&type=artist&limit=100&method=indexed".format(quote(artist))

        url = urllib.parse.urljoin(baseURL, extra)         
                    
        ## Download data
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False

        
        ## Parse data
        bsdata = getHTML(data)
        
        artistDB  = {}

        tables = bsdata.findAll("table")
        for table in tables:
            ths = table.findAll("th")
            headers = [x.text for x in ths]
            trs = table.findAll("tr")
            for tr in trs[1:]:
                tds    = tr.findAll("td")
                name   = tds[0].find('a').text
                href   = tds[0].find('a').attrs['href']
            
                if artistDB.get(href) is None:
                    artistDB[href] = {"N": 0, "Name": name}
                artistDB[href]["N"] += 1
        
    
        if self.debug:
            print("Found {0} artists".format(len(artistDB)))
                
        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1

            discID   = self.dutils.getArtistID(href)
            
            uuid = href.split('/')[-1]

            m = md5()
            for val in uuid.split("-"):
                m.update(val.encode('utf-8'))
            hashval = m.hexdigest()
            discID  = str(int(hashval, 16))

            url      = self.getArtistURL(href)
            savename = self.getArtistSavename(discID)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url)
            
            if isFile(savename):
                continue

            self.downloadArtistURL(url, savename)

In [ ]:
am = artistsMusicBrainz(debug=True)
#am.searchForArtist("Niel Diamond")

In [ ]:
from artistLM import artistLM
from discogsUtils import lastfmUtils

class artistsLastFM(artistsBase):
    def __init__(self, debug=False):
        self.db     = "LastFM"
        self.disc   = discogs(self.db.lower())
        self.artist = artistLM(self.disc)
        self.dutils = lastfmUtils()
        self.debug  = debug
        super().__init__(self.db, self.disc, self.artist, self.dutils, debug=debug)

        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, artistRef, page=1):
        #print("getArtistURL(",artistRef,")")
        if artistRef.startswith("http"):
            url = artistRef
        else:
            baseURL = self.disc.discogURL
            if artistRef.startswith("/") is True:
                #print("Join", end="\t")
                url     = urllib.parse.urljoin(baseURL, quote(artistRef[1:]))
                #print(url)
            else:
                url     = urllib.parse.urljoin(baseURL, quote(artistRef))
            #print(url)
                
            if url.endswith("/") is False:
                url     = "{0}{1}".format(url, "/+albums?order=release_date")
            else:
                url     = "{0}{1}".format(url, "+albums?order=release_date")
                
            #print(url)
        
        if isinstance(page, int) and page > 1:
            pageURL = "&page={0}".format(page)
            url = "{0}{1}".format(url, pageURL)
        return url

        
    ##################################################################################################################
    # Search Functions
    ##################################################################################################################
    def searchForArtist(self, artist):
        print("\n\n===================== Searching For {0} =====================".format(artist))
        baseURL = self.disc.discogSearchURL
        url = urllib.parse.urljoin(baseURL, "{0}{1}".format("artists?q=", quote(artist)))
                    
            
        ## Download data
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False

        
        ## Parse data
        bsdata = getHTML(data)
        
        artistDB  = {}
        
        uls = bsdata.findAll("ul", {"class": "artist-results"})
        for ul in uls:
            lis = ul.findAll("li", {"class": "artist-result"})
            for li in lis:
                h4 = li.find("h4")
                if h4 is None:
                    raise ValueError("No h4 in list")
                ref = h4.find('a')
                if ref is None:
                    raise ValueError("No ref in h4")

                name = ref.attrs['title']
                url  = ref.attrs['href']
                artistID = self.dutils.getArtistID(name)
        
                #print(name,'\t',url,'\t',artistID)
                if artistDB.get(url) is None:
                    artistDB[url] = {"N": 0, "Name": name}
                artistDB[url]["N"] += 1
        
    
        if self.debug:
            print("Found {0} artists".format(len(artistDB)))
                
        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1
        
            name     = hrefData["Name"]
            discID   = self.dutils.getArtistID(name)
            url      = self.getArtistURL(href)
            savename = self.getArtistSavename(discID)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url)
            
            if isFile(savename):
                continue

            self.downloadArtistURL(url, savename)
                
    
    def assertDBModValExtraData(self, modVal, maxPages=None):
        mulArts             = multiartist()        
        
        print("assertDBModValExtraData(",modVal,")")
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
        dbdata  = getFile(dbname)
        nerrs = 0
        
        for artistID,artistData in dbdata.items():
            pages = artistData.pages
            if pages.more is True:
                npages = pages.tot
                if maxPages is not None:
                    npages = min([npages, maxPages])
                artistRef = artistData.url.url
                print(artistID,'\t',artistData.artist.name)
                multiValues = mulArts.getArtistNames(artistData.artist.name)
                if len(multiValues) > 1:
                    print("\tNot downloading multis: {0}".format(multiValues.keys()))
                    continue
                for p in range(2, npages+1):
                    url      = self.getArtistURL(artistRef, p)
                    savename = self.getArtistSavename(artistID, p)
                    print(artistID,'\t',url,'\t',savename)
                    if not isFile(savename):
                        self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
                        sleep(3)

In [ ]:
al = artistsLastFM(debug=True)

In [ ]:
#al.searchForArtist("Neil Diamond")

In [ ]:
from artistRC import artistRC
from discogsUtils import rockcornerUtils

class artistsRockCorner(artistsBase):
    def __init__(self, debug=False):
        self.db     = "RockCorner"
        self.disc   = discogs(self.db.lower())
        self.artist = artistRC(self.disc)
        self.dutils = rockcornerUtils()
        self.debug  = debug
        super().__init__(self.db, self.disc, self.artist, self.dutils, debug=debug)

        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, artistRef, page=1):
        if artistRef.startswith("http"):
            url = artistRef
        else:
            baseURL = self.disc.discogURL
            if artistRef.startswith("/") is True:
                #print("Join", end="\t")
                url     = urllib.parse.urljoin(baseURL, quote(artistRef[1:]))
                #print(url)
            else:
                url     = urllib.parse.urljoin(baseURL, quote(artistRef))
                        
        if isinstance(page, int) and page > 1:
            pageURL = "&page={0}".format(page)
            url = "{0}{1}".format(url, pageURL)
        return url

        
    ##################################################################################################################
    # Search Functions
    ##################################################################################################################
    def searchForArtist(self, artist):

        print("\n\n===================== Searching For {0} =====================".format(artist))
        baseURL = self.disc.discogSearchURL
        url = urllib.parse.urljoin(baseURL, "{0}{1}".format("search?q=", quote(artist)))
                    

        ## Download data
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False

        
        ## Parse data
        bsdata = getHTML(data)
        
        artistDB  = {}

        songs = bsdata.findAll("article", {"class": "bgl0"}) + bsdata.findAll("article", {"class": "bgl1"})
        for i,song in enumerate(songs):
            label     = song.find("label")
            if label is None:
                continue
            name      = label.text
            ref       = song.find("a").attrs['href']
            artistURL = "/".join(ref.split("/")[:2])

            #print(name,'\t',url,'\t',artistID)
            if artistDB.get(artistURL) is None:
                artistDB[artistURL] = {"N": 0, "Name": name}
            artistDB[artistURL]["N"] += 1


        if self.debug:
            print("Found {0} artists".format(len(artistDB)))

        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1

            name     = hrefData["Name"]
            discID   = self.dutils.getArtistID(name)
            url      = self.getArtistURL(href)
            savename = self.getArtistSavename(discID)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url)
            
            if isFile(savename):
                continue

            self.downloadArtistURL(url, savename)

In [ ]:
ar = artistsRockCorner(debug=True)

In [ ]:
#ar.searchForArtist("Neil Diamond")

In [ ]:
from artistRM import artistRM
from discogsUtils import rateyourmusicUtils

class artistsRateYourMusic(artistsBase):
    def __init__(self, debug=False):
        self.db     = "RateYourMusic"
        self.disc   = discogs(self.db.lower())
        self.artist = artistRM(self.disc)
        self.dutils = rateyourmusicUtils()
        self.debug  = debug
        super().__init__(self.db, self.disc, self.artist, self.dutils, debug=debug)

        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, artistRef, page=1):
        baseURL = self.disc.discogURL
        url     = urllib.parse.urljoin(baseURL, artistRef)
        return url

        
    ##################################################################################################################
    # Search Functions
    ##################################################################################################################
    def searchForArtist(self, artist):
        return

In [ ]:
am = artistsRateYourMusic(debug=True)

In [ ]:
from artistCL import artistCL
from discogsUtils import cdandlpUtils

class artistsCDandLP(artistsBase):
    def __init__(self, debug=False):
        self.db     = "CDandLP"
        self.disc   = discogs(self.db.lower())
        self.artist = artistCL(self.disc)
        self.dutils = cdandlpUtils()
        self.debug  = debug
        super().__init__(self.db, self.disc, self.artist, self.dutils, debug=debug)

        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, artistRef, page=1):
        #print("getArtistURL(",artistRef,")")
        if artistRef.startswith("http"):
            url = artistRef
        else:
            baseURL = self.disc.discogURL
            if artistRef.startswith("/") is True:
                #print("Join", end="\t")
                url     = urllib.parse.urljoin(baseURL, quote(artistRef[1:]))
                #print(url)
            else:
                url     = urllib.parse.urljoin(baseURL, quote(artistRef))
        
        if isinstance(page, int) and page > 1:
            pageURL = "&page={0}".format(page)
            url = "{0}{1}".format(url, pageURL)
        return url

        
    ##################################################################################################################
    # Search Functions
    ##################################################################################################################
    def searchForArtist(self, artist):
        print("\n\n===================== Searching For {0} =====================".format(artist))
        baseURL = self.disc.discogSearchURL
        url = urllib.parse.urljoin(baseURL, "{0}{1}".format("?q=", quote(artist)))


        ## Download data
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False

        
        ## Parse data
        bsdata = getHTML(data)
        
        artistDB  = {}
        
        descrs = bsdata.findAll("div", {"class": "listingDescription"})
        for descr in descrs:
            refs = descr.findAll("a", {"class": "listingTitle"})
            for ref in refs:
                url      = ref.attrs['href']
                fullurl  = "/".join(url.split("/")[:-4])
                fullurl  = "{0}/artist".format(fullurl)
                artistID = self.dutils.getArtistID(fullurl)
                if artistDB.get(fullurl) is None:
                    artistDB[fullurl] = {"N": 0, "ID": artistID}
                artistDB[fullurl]["N"] += 1
        
    
        if self.debug:
            print("Found {0} artists".format(len(artistDB)))
                
        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1
        
            discID   = hrefData["ID"]
            url      = href
            savename = self.getArtistSavename(discID)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url,'\t',savename)
            
            if isFile(savename):
                continue

            self.downloadArtistURL(url, savename)

In [ ]:
ac = artistsCDandLP(debug=True)

In [ ]:
#ac.searchForArtist("Neil Diamond")

In [ ]:
from artistCL import artistCL
from discogsUtils import acebootlegsUtils

class artistsAceBootlegs(artistsBase):
    def __init__(self, debug=False):
        self.db     = "AceBootlegs"
        self.disc   = discogs(self.db.lower())
        self.artist = artistAB(self.disc)
        self.dutils = acebootlegsUtils()
        self.debug  = debug
        super().__init__(self.db, self.disc, self.artist, self.dutils, debug=debug)

        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, artistRef, page=1):
        baseURL = self.disc.discogURL
        url     = urllib.parse.urljoin(baseURL, artistRef)
        return url
    
    def getMainSavename(self):
        artistDir = self.disc.getArtistsDir()
        savename  = setFile(artistDir, "main.p")
        return savename
    
    
    ##################################################################################################################
    # Main
    ##################################################################################################################
    def downloadMainArtists(self, force=False, debug=False, sleeptime=2):        
        savename = self.getMainSavename()
        
        ## Parse data
        bsdata = getHTML(savename)        
        artistDB  = {}
        
        
        ## Find and Download Artists
        categories = bsdata.find("div", {"class": "sidebar-widget widget_categories"})
        if categories is None:
            raise ValueError("Cannot find categories!")
        uls = categories.findAll("ul")
        for ul in uls:
            lis = ul.findAll("li")
            for i, li in enumerate(lis):
                try:
                    catitem = li.attrs["class"][1]
                except:
                    raise ValueError("Cannot find list class item: {0}".format(li))
                ref  = li.find("a")
                if ref is None:
                    raise ValueError("Cannot find list link!")
                try:
                    href = ref.attrs['href']
                except:
                    raise ValueError("Cannot find list href!")

                # check for artist
                artistName = href.split('/')[-2]
                try:
                    int(artistName)
                    continue
                except:
                    if artistName.find("parent-category-ii") == -1:
                        pass
                    else:
                        continue

                # get artist ID
                artistID = catitem.split('-')[-1]
                try:
                    int(artistID)
                except:
                    continue


                if force is False and isFile(savename):
                    print("{0} exists.".format(savename))
                    continue
            
                url = href
                savename = self.getArtistSavename(artistID)
                print(i,'\t',artistID,'\t',artistName,'\t',savename)
                self.downloadArtistURL(url=url, savename=savename, parse=False)

                
        
        

        
    ##################################################################################################################
    # Search Functions
    ##################################################################################################################
    def searchForArtist(self, artist):
        return

In [ ]:
aa = artistsAceBootlegs(debug=True)

In [ ]:
from artistDP import artistDP
from discogsUtils import datpiffUtils

class artistsDatPiff(artistsBase):
    def __init__(self, debug=False):
        self.db     = "DatPiff"
        self.disc   = discogs(self.db.lower())
        self.artist = artistDP(self.disc)
        self.dutils = datpiffUtils()
        self.debug  = debug
        super().__init__(self.db, self.disc, self.artist, self.dutils, debug=debug)

        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, artistRef, page=1):
        baseURL = self.disc.discogURL
        url     = urllib.parse.urljoin(baseURL, artistRef)
        return url
    

    ##################################################################################################################
    # Search Functions
    ##################################################################################################################
    def searchForArtist(self, artist):
        print("\n\n===================== Searching For {0} =====================".format(artist))
        baseURL = self.disc.discogSearchURL
        
        baseURL = "https://www.datpiff.com/"
        extra   = "mixtapes-search?criteria={0}&sort=relevance".format(quote(artist))

        url = urllib.parse.urljoin(baseURL, extra)
        print("\tSearch URL: {0}".format(url))
                    
        ## Download data
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False

        
        ## Parse data
        bsdata = getHTML(data)
        
        artistDB  = []

        contentdivs = bsdata.findAll("div", {"class": "contentItem"})
        for i,contentdiv in enumerate(contentdivs):
            artistDiv = contentdiv.find("div", {"class": "artist"})
            if artistDiv is None:
                continue
            artistName = artistDiv.text

            albumDiv = contentdiv.find("div", {"class": "title"})
            if albumDiv is None:
                continue
            albumName = albumDiv.text
            try:
                albumURL  = albumDiv.find("a").attrs['href']
            except:
                albumURL  = None
                
            #artistID = self.discogsUtils.getArtistID(artistName)

            artistDB.append({"ArtistName": artistName, "AlbumName": albumName, "AlbumURL": albumURL})
        

        artistID = self.discogsUtils.getArtistID(artist)
        page     = 1
        savename = self.getArtistSavename(artistID, page)
        while isFile(savename):
            page += 1
            savename = self.getArtistSavename(artistID, page)
        print("Saving {0} new artist media to {1}".format(len(artistDB), savename))
        saveJoblib(data=artistDB, filename=savename, compress=True)

In [ ]:
ad = artistsDatPiff(debug=True)

In [ ]:
from artistMS import artistMS
from discogsUtils import musicstackUtils
from searchUtils import findPatternExt
from fileUtils import getBaseFilename


class artistsMusicStack(artistsBase):
    def __init__(self, debug=False):
        self.db     = "MusicStack"
        self.disc   = discogs(self.db.lower())
        self.artist = artistMS(self.disc)
        self.dutils = musicstackUtils()
        self.debug  = debug
        super().__init__(self.db, self.disc, self.artist, self.dutils, debug=debug)

        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, artistRef, page=1):
        baseURL = self.disc.discogURL
        url     = urllib.parse.urljoin(baseURL, artistRef)
        return url
    

    ##################################################################################################################
    # Search Functions
    ##################################################################################################################
    def searchForArtist(self, artist=None):
        files = findPatternExt("/Users/tgadfort/Downloads/", pattern="MusicStack", ext=".html")
        for ifile in files:
            fname  = getBaseFilename(ifile)
            artist = fname[:fname.find(" Vinyl Records")]
            self.parseSearchForArtist(artist=artist, ifile=ifile)
    
    def parseSearchForArtist(self, artist, ifile):

        ## Parse data
        bsdata = getHTML(ifile)
        
        artistDB  = []

        tables = bsdata.findAll("table")
        if self.debug:
            print("Found {0} tables".format(len(tables)))
        for i,table in enumerate(tables):
            trs = table.findAll("tr")
            jj = 0
            headers = None
            values  = []
            for j,tr in enumerate(trs):
                tds  = tr.findAll("td")
                #print('\t',len(tds))
                #tds  = [list(td.strings) for td in tds]
                if len(tds) == 8:
                    if jj == 0:
                        headers = tds
                        if self.debug:
                            print("  Found Header: {0}".format(headers))
                    else:
                        values.append(tds)
                        if self.debug:
                            print("  Found Value: {0}".format(tds))

                    jj += 1

            if headers is not None:
                keys = []
                for header in headers:
                    b = header.find("b")
                    if b is None:
                        keys.append(str(len(keys)))
                    else:
                        txt = b.text.strip()
                        keys.append(txt)
                        
                if self.debug:
                    print("  Keys: {0}".format(keys))
                    print("  Values: {0}".format(len(values)))


                for value in values:
                    value = [x for ix,x in enumerate(value) if keys[ix] in ["Artist", "Title"]]
                    album = dict(zip(["Artist", "Title"], value))
                    album["Artist"] = album["Artist"].text
                    album["Album"]  = {"Name": album["Title"].text}
                    try:
                        album["Album"]["URL"] = album["Title"].find("a").attrs['href']
                    except:
                        album["Album"]["URL"] = None
                    del album["Title"]
                    artistDB.append({"ArtistName": album["Artist"], "AlbumName": album["Album"]["Name"], "AlbumURL": album["Album"]["URL"]})
                break

        artistID = self.dutils.getArtistID(artist)
        page     = 1
        savename = self.getArtistSavename(artistID, page)
        while isFile(savename) and False:
            page += 1
            savename = self.getArtistSavename(artistID, page)
        print("Saving {0} new artist media to {1}".format(len(artistDB), savename))
        saveJoblib(data=artistDB, filename=savename, compress=True)

In [ ]:
am = artistsMusicStack(debug=False)
am.searchForArtist()